In [25]:
import pandas as pd
import numpy as np

In [26]:
df = pd.DataFrame({"a":np.random.randn(5),"b":np.random.randn(5)})

In [30]:
s1 = pd.Series({"a":123,"b":456})
s2 = pd.Series({"a":123,"b":456})

In [31]:
pd.concat([s1,s2])

a    123
b    456
a    123
b    456
dtype: int64

In [1]:
str_tuple = "(1,23)"
tuple(str_tuple)

('(', '1', ',', '2', '3', ')')